In [5]:
import numpy as np

In [6]:
class Conv3x3:
  # A Convolution layer using 3x3 filters.

    def __init__(self, num_filters):
        self.num_filters = num_filters

        self.filters = np.random.randn(num_filters, 3, 3) / 9
    def iterate_regions(self, image):
   
    
        h, w = image.shape

        for i in range(h - 2):
              for j in range(w - 2):
                    im_region = image[i:(i + 3), j:(j + 3)]
                    yield im_region, i, j

    def forward(self, input):
  
        self.last_input = input

        h, w = input.shape
        output = np.zeros((h - 2, w - 2, self.num_filters))

        for im_region, i, j in self.iterate_regions(input):
              output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

        return output

    def backprop(self, d_L_d_out, learn_rate):
   
        d_L_d_filters = np.zeros(self.filters.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
              for f in range(self.num_filters):
                    d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

   
        self.filters -= learn_rate * d_L_d_filters

        return None

In [10]:

class MaxPool2:
  # A Max Pooling layer using a pool size of 2.

    def iterate_regions(self, image):
    
        h, w, _ = image.shape
        new_h = h // 2
        new_w = w // 2

        for i in range(new_h):
              for j in range(new_w):
                im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield im_region, i, j

    def forward(self, input):
   
        self.last_input = input

        h, w, num_filters = input.shape
        output = np.zeros((h // 2, w // 2, num_filters))

        for im_region, i, j in self.iterate_regions(input):
              output[i, j] = np.amax(im_region, axis=(0, 1))

        return output

    def backprop(self, d_L_d_out):
    
        d_L_d_input = np.zeros(self.last_input.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
                  h, w, f = im_region.shape
                  amax = np.amax(im_region, axis=(0, 1))

                  for i2 in range(h):
                        for j2 in range(w):
                              for f2 in range(f):
                                    if im_region[i2, j2, f2] == amax[f2]:
                                          d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]

        return d_L_d_input